In [1]:
import pandas as pd
import numpy as np
from faker import Faker
from collections import defaultdict
from sqlalchemy import create_engine
import datetime
import random
import re


# Fake data creation

In [2]:
from numpy import integer


fake=Faker()

fake_data = defaultdict(list)

row_number=int(input("Please write the number you wish to see the row size, DON'T CRASH ME!"))



In [3]:
# Creating sender_id as much as the given input
for i in range(row_number):
    fake_data['sender_id'].append(fake.bothify(text='####'))

sender_id=fake_data["sender_id"]

#Creating other futures
manipulated_receiver_id=[]
amount=[]
timestamp=[]

In [4]:
for idx in range(len(fake_data['sender_id'])): #For loop for creating random values in features

    temp_sender_val=sender_id[idx]
    temp_receiver_list=[]
    temp_amount_list=[]
    temp_timestamp_list=[]
    
    #Creating random relationship on two nodes
    for j in range(random.randint(1,10)): 
        
        #Choosing random sender_id and adding to reciever_id
        rnd_reciever_id=''.join(random.choices(sender_id)) 
        
        #If sender and receiver feature value same break from the loop
        if (temp_sender_val== rnd_reciever_id):
            continue

        #Creating non-existed features
        temp_receiver_list.append(rnd_reciever_id)
        temp_amount_list.append(fake.pyfloat(right_digits=2,positive=True,min_value=2.00,max_value=10000.00))
        temp_timestamp_list.append(fake.date_time_ad(start_datetime=datetime.datetime(2022,9,14)))
    
    #Check the size of the receiver_id compare it with sender_id
    if len(manipulated_receiver_id)==len(fake_data['sender_id']):
            break


    #Appending every feature value to list
    manipulated_receiver_id.append(temp_receiver_list)
    amount.append(temp_amount_list)
    timestamp.append(temp_timestamp_list)
        

In [5]:
#Ejecting fake/manipulated data do Dataframe

fake_data['receiver_id']=manipulated_receiver_id
fake_data['amount']=amount
fake_data['timestamp']=timestamp
print(len(fake_data['amount']))
df=pd.DataFrame(fake_data)
df=df[['sender_id','receiver_id','amount','timestamp']]
df

10


,sender_id,receiver_id,amount,timestamp
0,9718,"[0234, 7047, 6673, 6091, 0841, 6673]","[9506.3, 4161.95, 7939.8, 9866.85, 4344.94, 71...","[2022-10-08 21:24:00, 2022-10-20 06:36:58, 202..."
1,4132,"[3922, 6673, 0841, 6673, 4574, 0234, 7047, 971...","[7833.45, 8556.83, 6274.4, 5714.17, 331.36, 57...","[2022-09-16 21:54:08, 2022-10-17 15:20:33, 202..."
2,0841,"[4574, 6091, 4132, 4132, 7047, 7047, 3922, 3922]","[6463.3, 8331.83, 1841.29, 1445.53, 5891.66, 8...","[2022-10-01 01:33:21, 2022-10-22 04:49:53, 202..."
3,6091,"[0234, 3922, 9718, 9223, 0841, 0234, 0234, 4574]","[8743.91, 1083.0, 1835.34, 4033.26, 3298.62, 4...","[2022-09-16 12:43:52, 2022-10-22 18:52:35, 202..."
4,3922,"[6091, 0234, 4574, 4132, 4132, 6673, 4574, 457...","[258.86, 4026.7, 8326.47, 5449.87, 6390.28, 61...","[2022-09-21 17:15:37, 2022-10-13 06:37:08, 202..."
5,0234,"[4132, 4132, 3922, 0841]","[1857.22, 9553.44, 9909.18, 3764.78]","[2022-09-25 22:06:11, 2022-10-15 12:48:08, 202..."
6,6673,"[0841, 9718, 7047]","[4543.94, 6982.21, 2181.31]","[2022-11-01 08:13:06, 2022-10-11 11:31:08, 202..."
7,4574,"[0841, 6673, 6673, 3922, 6673, 0841, 9718, 0841]","[3827.31, 1164.54, 771.54, 803.93, 7466.6, 688...","[2022-11-02 17:29:41, 2022-10-05 13:04:52, 202..."
8,7047,"[6673, 0841, 0234, 9718, 4132, 3922, 0841, 4574]","[5917.94, 1267.4, 5440.92, 7482.86, 7144.59, 2...","[2022-09-14 19:56:32, 2022-10-02 20:41:52, 202..."
9,9223,"[0234, 4132]","[6106.0, 4561.84]","[2022-09-21 00:37:22, 2022-10-08 18:54:07]"


# Neo4j

In [6]:
from neo4j import GraphDatabase

In [7]:
transaction_info=df.values.tolist()

In [9]:
uri = "neo4j://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "test"))

#Deleting existing values and relationships
def clear_db(tx):
    tx.run("MATCH(n) Detach Delete n")
#Creating customer node
def create_customer(tx, customer_id):
    tx.run("CREATE (c1:customer {name: $name, customer_id: $id})",name='customer', id=customer_id)

#Creating relationship between customer nodes
def create_transaction(tx,customer_id,other_customer_id,amount,timestamp):
    tx.run("MATCH (c1:customer),(c2:customer) "
    "Where c1.customer_id= $id and c2.customer_id= $otherid "
    "Create (c1)-[t:Transaction{amount: $amount, time: $timestamp}]->(c2)",
    id=customer_id,otherid=other_customer_id,amount=amount,timestamp=timestamp)


with driver.session() as session:

    session.execute_write(clear_db)

    for i in range(len(transaction_info)):
        session.write_transaction(create_customer, str(transaction_info[i][0]))

    
    for element in transaction_info:
        for block in range(1,len(element)):
            for j in range(len(element[block])):

                session.write_transaction(create_transaction,element[0],element[1][j],element[2][j],element[3][j])

driver.close()

/tmp/ipykernel_10930/2877050615.py:24: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_customer, str(transaction_info[i][0]))
/tmp/ipykernel_10930/2877050615.py:31: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_transaction,element[0],element[1][j],element[2][j],element[3][j])
